In [ ]:
# STEP 1: Install dependencies
!pip install langchain youtube-transcript-api sentence-transformers faiss-cpu

# STEP 2: Imports
from youtube_transcript_api import YouTubeTranscriptApi
from langchain.text_splitter import CharacterTextSplitter
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# STEP 3: Get YouTube transcript
def get_transcript(video_id):
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        return " ".join([entry['text'] for entry in transcript])
    except Exception as e:
        return f"Error fetching transcript: {e}"

# Try with this video (change to your own if needed)
video_id = "ATlila3e9dM"  # <-- replace with actual video ID
raw_text = get_transcript(video_id)

# STEP 4: Chunk the transcript
splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=100)
docs = splitter.create_documents([raw_text])

# STEP 5: Create embeddings using SentenceTransformers
model = SentenceTransformer('all-MiniLM-L6-v2')

# Access the text from the Document objects using .page_content
embeddings = model.encode([doc.page_content for doc in docs], show_progress_bar=True)

# STEP 6: Create FAISS index
dim = embeddings.shape[1]  # Embedding dimension
index = faiss.IndexFlatL2(dim)  # L2 distance metric
index.add(np.array(embeddings))  # Add embeddings to FAISS index

# STEP 7: Create a function for querying the FAISS index
def query_faiss(query, k=5):
    query_embedding = model.encode([query])
    distances, indices = index.search(query_embedding, k)
    return [(docs[i].page_content, distances[0][idx]) for idx, i in enumerate(indices[0])]

# STEP 8: Query the system for a summary
query = "Summarize this video in a short paragraph."
results = query_faiss(query, k=3)  # Get top 3 most relevant chunks
summary = " ".join([result[0] for result in results])
print("🔍 Summary:\n", summary)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

🔍 Summary:
 Look at this baby. So gentle, so innocent. You'd think this child would grow up to become a good man, an honest man. Well, think again. *alarm sounds* Hitler was born Adolphus Hitler in 1889 in a small town in Austria-Hungary. His father, Alois Schicklgruber, was born out of wedlock, but eventually changed his name to that of his stepfather, becoming Alois Hitler. Alois was a mid-level Austrian customs officer-- not really rolling in cash, but certainly rolling in women. He married a rich, older lady, but then immediately started having affairs, including one with a much younger house servant. A few years later, he left his sick wife to be with his mistress, but since the Catholic Church didn't allow divorce at the time, he couldn't marry her. So he waited for his old wife to die and had a child in the meantime. Then his wife died, so he married his mistress and had another child, but then his new wife got sick, so he employed his much, much younger cousin Clara to take car

In [ ]:
!pip install -U langchain langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00
